In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import scipy
from scipy.spatial import distance
import os
from os import listdir
from os.path import isfile, join

In [2]:
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten,Dense,GlobalAveragePooling2D,Lambda,add, Dropout, Activation,Concatenate,BatchNormalization
import numpy as np
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras import backend as K

Using TensorFlow backend.


In [3]:
import mtcnn
from mtcnn.mtcnn import MTCNN
model2 = MTCNN(min_face_size = 10)
from inception_resnet_v1 import *
model = InceptionResNetV1()
model.load_weights('facenet_weights.h5')

In [4]:
def l2_normalize(x, axis=-1, epsilon=1e-10):
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [5]:
def preprocess(x):
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std_adj
    return y

In [6]:
def faceinfo(image,margin =10):
    person_image1 = cv2.imread(image)
    person_image = cv2.cvtColor(person_image1, cv2.COLOR_BGR2RGB)
    face_info = model2.detect_faces(person_image)
    for face in face_info:
        if face['confidence']>0.98:
            x,y,w,h = face['box']
            x,y = abs(x),abs(y)
            face = person_image1[y-margin//2:y+h+margin//2,x-margin//2:x+w+margin//2,:]
            roi = cv2.resize(face, (160,160))
            break
        else:
            pass
    return roi

In [7]:
def add_photo(image_path):
    name1 = str(input('Enter the name of person'))
    number = int(input('Enter the sr. number '))
    im = faceinfo(image_path)
    cv2.imwrite('crop_faces2/'+name1+ str(number) + '.jpg',im)
    im2 = np.expand_dims(im, axis=0)
    im3 = l2_normalize(model.predict(preprocess(im2)))
    all_people_faces[name + str(number)] = im3
    print('Done')

In [8]:
import pickle

In [9]:
all_people_faces = pickle.load(open('all_people_faces.pkl','rb'))

In [10]:
with open('all_people_faces' + '.pkl', 'wb') as f:
    pickle.dump(all_people_faces, f, pickle.HIGHEST_PROTOCOL)

In [11]:
def match_face(image_address,margin = 10):
    image = cv2.imread(image_address)
    image1 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)    
    face_info = model2.detect_faces(image1)
    for face in face_info:
        if face['confidence']>0.98:
            x,y,w,h = face['box']
            x,y = abs(x),abs(y)
            face = image1[y-margin//2:y+h+margin//2,x-margin//2:x+w+margin//2,:]
            roi = cv2.resize(face, (160,160))
            roi = np.expand_dims(roi, axis=0)
            captured_representation = l2_normalize(model.predict(preprocess(roi))[0])
        
            cv2.rectangle(image, (x,y), (x+w,y+h), (0,0,255), 1)
            cv2.rectangle(image,(x,y-20),(x+w+40,y),(0,255,0),-1)
            for i in all_people_faces:
                person_name = i
                representation = all_people_faces[i]

                similarity =  distance.euclidean(representation, captured_representation)
                found = 0
                if(similarity < 0.82):
                    cv2.putText(image, person_name[:-1], (int(x), int(y-5)), 0, 0.6, (0, 0, 255), 2)
                    found = 1
                    break
            if(found == 0):
                cv2.putText(image, 'unknown', (int(x), int(y-5)), 0, 0.6, (255, 0, 0), 2)
            print(similarity)
        else:
            pass
    
    cv2.imshow('Face Recognition', image)
    cv2.waitKey(0)    
    cv2.destroyAllWindows()

In [17]:
match_face('Test Images2/50.jpg')

0.6357426047325134
1.4085180759429932
0.44825151562690735
